In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# Definir la función de pérdida
def loss_func(theta):
    x, y = theta
    R = np.sqrt(x**2 + y**2)
    return -np.sin(R)

In [14]:
# Definir el gradiente de la función de pérdida
def evaluate_gradient(theta):
    x, y = theta
    R = np.sqrt(x**2 + y**2) + 1e-8  # Agregar epsilon para evitar división por cero
    grad_x = -np.cos(R) * (x / R)
    grad_y = -np.cos(R) * (y / R)
    return np.array([grad_x, grad_y])

In [15]:
# 1. Gradiente Descendente
def gradient_descent(theta_init, loss_func, evaluate_gradient, eta, epochs):
    theta = theta_init.copy()
    for _ in range(epochs):
        gradient = evaluate_gradient(theta)
        theta -= eta * gradient
    return theta

In [16]:
# 2. Gradiente Descendente Estocástico (SGD)
def stochastic_gradient_descent(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs):
    theta = theta_init.copy()
    for _ in range(epochs):
        np.random.shuffle(data_train)
        for x_i in data_train:
            gradient = evaluate_gradient(theta)
            theta -= eta * gradient
    return theta

In [17]:
# 3. RMSPROP
def rmsprop(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs, decay=0.9, epsilon=1e-8):
    theta = theta_init.copy()
    Eg2 = np.zeros_like(theta)
    for _ in range(epochs):
        np.random.shuffle(data_train)
        for x_i in data_train:
            gradient = evaluate_gradient(theta)
            Eg2 = decay * Eg2 + (1 - decay) * gradient**2
            theta -= (eta / (np.sqrt(Eg2) + epsilon)) * gradient
    return theta

In [18]:
# 4. Adam
def adam(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs, beta1=0.9, beta2=0.999, epsilon=1e-8):
    theta = theta_init.copy()
    m = np.zeros_like(theta)
    v = np.zeros_like(theta)
    t = 0
    for _ in range(epochs):
        np.random.shuffle(data_train)
        for x_i in data_train:
            t += 1
            gradient = evaluate_gradient(theta)
            m = beta1 * m + (1 - beta1) * gradient
            v = beta2 * v + (1 - beta2) * gradient**2
            m_hat = m / (1 - beta1**t)
            v_hat = v / (1 - beta2**t)
            theta -= (eta / (np.sqrt(v_hat) + epsilon)) * m_hat
    return theta

In [19]:
# Configuración del experimento
num_iterations = 10000  # Número de iteraciones del experimento
methods = ['Gradient Descent', 'Stochastic Gradient Descent', 'RMSPROP', 'Adam']
results = {'Method': [], 'Distance': []}

# Datos de entrenamiento (no se usan realmente en esta función, pero los incluimos para consistencia)
n_points = 1  # Solo necesitamos un punto para SGD, RMSPROP y Adam
data_train = [np.array([0, 0])]  # Punto ficticio

# Parámetros comunes
eta = 0.01  # Tasa de aprendizaje
epochs = 1  # Número de épocas (iteraciones internas)

In [20]:

# Ejecutar el experimento
for _ in range(num_iterations):
    # Generar un punto inicial aleatorio en el rango [-6.5, 6.5]
    theta_init = np.random.uniform(-6.5, 6.5, 2)
    
    # Almacenar las distancias para cada método en esta iteración
    distances = {}
    
    # Gradiente Descendente
    theta_gd = gradient_descent(theta_init, loss_func, evaluate_gradient, eta, epochs=100)
    dist_gd = np.linalg.norm(theta_gd - np.array([0, 0]))
    distances['Gradient Descent'] = dist_gd
    
    # Gradiente Descendente Estocástico
    theta_sgd = stochastic_gradient_descent(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs=100)
    dist_sgd = np.linalg.norm(theta_sgd - np.array([0, 0]))
    distances['Stochastic Gradient Descent'] = dist_sgd
    
    # RMSPROP
    theta_rmsprop = rmsprop(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs=100)
    dist_rmsprop = np.linalg.norm(theta_rmsprop - np.array([0, 0]))
    distances['RMSPROP'] = dist_rmsprop
    
    # Adam
    theta_adam = adam(theta_init, data_train, loss_func, evaluate_gradient, eta, epochs=100)
    dist_adam = np.linalg.norm(theta_adam - np.array([0, 0]))
    distances['Adam'] = dist_adam
    
    # Determinar el método con la menor distancia en esta iteración
    best_method = min(distances, key=distances.get)
    
    # Almacenar los resultados
    results['Method'].append(best_method)
    results['Distance'].append(distances[best_method])

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results)

In [21]:
# Calcular la distancia promedio para cada método
average_distances = results_df.groupby('Method')['Distance'].mean()
print("Distancias promedio al punto (0,0) por método:")
print(average_distances)

# Contar la frecuencia con la que cada método fue el mejor
frequency_table = results_df['Method'].value_counts().reset_index()
frequency_table.columns = ['Method', 'Absolute Frequency']
frequency_table['Relative Frequency (%)'] = (frequency_table['Absolute Frequency'] / num_iterations) * 100

print("\nTabla de frecuencias (absoluta y relativa) de los mejores métodos:")
print(frequency_table)


Distancias promedio al punto (0,0) por método:
Method
Adam                3.438224
Gradient Descent    6.334317
RMSPROP             2.612417
Name: Distance, dtype: float64

Tabla de frecuencias (absoluta y relativa) de los mejores métodos:
             Method  Absolute Frequency  Relative Frequency (%)
0  Gradient Descent                5884                   58.84
1           RMSPROP                2937                   29.37
2              Adam                1179                   11.79
